In [4]:
import numpy as np
import random
import math
import time
from shapely.geometry import Polygon, MultiPolygon, Point, box
from shapely.affinity import scale, translate
from shapely import wkt
from shapely.ops import transform
from pyproj import Transformer
import folium
from folium.features import DivIcon
from IPython.display import display, clear_output

## Helper Functions

In [5]:
def calculateArea(poly_wkt_str):
    polygon = wkt.loads(poly_wkt_str)
    area_sq_degrees = polygon.area
    centroid = polygon.centroid
    lon, lat = centroid.x, centroid.y
    utm_zone = int((lon + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    # # Set up the transformation
    transformer = Transformer.from_crs("EPSG:4326", proj_string, always_xy=True)
    utm_polygon = transform(transformer.transform, polygon)
    area_sq_meters = utm_polygon.area
    area_sq_kilometers = area_sq_meters / 1_000_000  # Convert to square kilometers
    return float(format(area_sq_kilometers, '.2f'))

In [6]:
def displayOSM(i, polygon, bbox, area_percentage=None, uniform=None):
    # Create a map with the center based on the bounding box
    map_center = [(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2]
    m = folium.Map(location=map_center, zoom_start=10) # SWEDEN ZOOM 4
    
    if isinstance(polygon, str):
        # Assuming the string is WKT format
        try:
            shape = wkt.loads(polygon)
            geo_json = shape.__geo_interface__
        except Exception as e:
            print(f"Error parsing polygon {e}")
    else:
        # Assuming it's already in a format folium can use
        geo_json = polygon
        
        # Add the polygon to the map with a unique style
    folium.GeoJson(
        geo_json,
        style_function=lambda x, color=f'#4400ff': {
                'fillColor': color,
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5',
                'fillOpacity': 0.5,
        }
    ).add_to(m)

    north, south, east, west = bbox[3], bbox[1], bbox[0], bbox[2]
    # Add the bounding box to the map
    folium.Rectangle(bounds=[[north, west], [south, east]], color="red", fill=False).add_to(m)

    folium.map.Marker(
        [north-0.25, east-0.1],
        icon=DivIcon(
            icon_size=(550,56),
            icon_anchor=(0,0),
            html='<div style="font-size: 18pt; color: #4400ff;">points '+str(i)+' | area '+str(area_percentage)+' km² | uniformity '+str(uniform)+'</div>',
            )
        ).add_to(m)
    display(m)
    clear_output(wait=True)
    time.sleep(0.5)  # Pause to see the update
    # return m

## Generator Functions

In [7]:
def genPolygon_AreaFIXED(INPUT_BBOX, num_points=None, area_percentage=None, circum_radius_factor=None):
    """
    Generate different polygons within the INPUT_BBOX in EPSG:4326 - WGS 84
    Ranges from small area polygon to large polygons with large area in OGC WKT format
    
    Args:
        INPUT_BBOX: List [min_lon, min_lat, max_lon, max_lat]
        area_percentage: Optional float between 0 and 1, representing what percentage of the bbox area 
                        the polygon should occupy. If None, a random value is used.
    Returns:
        String: WKT format polygon
    """
    min_lon, min_lat, max_lon, max_lat = INPUT_BBOX
          
    # Center of bbox
    center_lon = (min_lon + max_lon) / 2
    center_lat = (min_lat + max_lat) / 2

    # Anywhere of bbox
    # center_lon = random.uniform(min_lon, max_lon)
    # center_lat = random.uniform(min_lat, max_lat)

    # Get bbox dimensions
    bbox_width = max_lon - min_lon
    bbox_height = max_lat - min_lat
    # Calculate polygon dimensions based on area percentage For simplicity, we'll create a rectangular polygon
    scale_factor = math.sqrt(area_percentage)
    poly_width = bbox_width * scale_factor
    poly_height = bbox_height * scale_factor
    
    # Calculate polygon corners
    half_width = poly_width / 2
    half_height = poly_height / 2
    
    # Create a more interesting polygon by adding some random points and perturbations
    points = []
    
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        
        # Base radius is determined by the desired area
        x_radius = half_width
        y_radius = half_height
        
        # Add some randomness but maintain approximate area
        radius_factor = random.uniform(0.5, 1.0) # How uniform are the points (0.1 = random,  1.0 = uniform)
        
        x = center_lon + math.cos(angle) * x_radius * radius_factor
        y = center_lat + math.sin(angle) * y_radius * radius_factor
        
        # Ensure points are within the bbox
        x = max(min(x, max_lon), min_lon)
        y = max(min(y, max_lat), min_lat)
        
        points.append((x, y))
    
    # Close the polygon by adding the first point at the end
    points.append(points[0])
    
    # Convert to WKT format
    wkt = "POLYGON (("
    wkt += ", ".join([f"{p[0]} {p[1]}" for p in points])
    wkt += "))"

    # Double-check OGC compliance
    # is_valid, message = validate_ogc_polygon(wkt)
    # if not is_valid:
    #     # Try to fix the polygon
    #     wkt = fix_ogc_polygon(wkt)
    return wkt

### Gen 1

In [8]:
def genPolygonExp(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR):
    """
    Generate a polygon with specified area in square kilometers
    
    Parameters:
    INPUT_BBOX: WKT string representing the bounding box
    NUM_OF_POINTS: Number of points in the polygon
    AREA_IN_km²: Target area in square kilometers
    UNIFORM_FACTOR: Controls shape regularity (0.0: very irregular, 1.0: perfectly regular)
    
    Returns:
    WKT string representing the generated polygon in EPSG:4326
    """
    # Parse the bounding box
    if type(INPUT_BBOX) == str:
        bbox = wkt.loads(INPUT_BBOX)
        bbox_bounds = bbox.bounds
        min_x, min_y, max_x, max_y = bbox_bounds
    else:
        min_x, min_y, max_x, max_y = INPUT_BBOX
        bbox = wkt.loads((box(*INPUT_BBOX, ccw=True)).wkt)
    
    # Compute the centroid for UTM zone determination
    centroid_x = (min_x + max_x) / 2
    centroid_y = (min_y + max_y) / 2
    
    # Determine UTM zone
    utm_zone = int((centroid_x + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    # Create transformers for WGS84 <-> UTM conversion
    transformer_to_utm = Transformer.from_crs("EPSG:4326", proj_string, always_xy=True)
    transformer_from_utm = Transformer.from_crs(proj_string, "EPSG:4326", always_xy=True)
    
    # Transform the bounding box to UTM
    utm_bbox = transform(transformer_to_utm.transform, bbox)
    utm_bounds = utm_bbox.bounds
    utm_min_x, utm_min_y, utm_max_x, utm_max_y = utm_bounds
    
    # Calculate the center point of the bounding box in UTM
    center_x = (utm_min_x + utm_max_x) / 2
    center_y = (utm_min_y + utm_max_y) / 2
    
    # Calculate the radius needed for the target area
    # For a regular polygon with n sides, area = (n/4) * r² * sin(2π/n)
    # Solving for r: r = sqrt(area / ((n/4) * sin(2π/n)))
    target_area_m2 = AREA_IN_km * 1_000_000  # Convert km² to m²
    
    # For a regular polygon, calculate the radius
    n = NUM_OF_POINTS
    regular_radius = math.sqrt(target_area_m2 / ((n/4) * math.sin(2 * math.pi / n)))
    
    # Generate points for the polygon
    angles = np.linspace(0, 2 * np.pi, NUM_OF_POINTS, endpoint=False)
    
    utm_points = []
    for angle in angles:
        # Start with perfectly regular distance (radius)
        base_distance = regular_radius
        
        # Add randomness based on the uniformity factor
        if UNIFORM_FACTOR < 1.0:
            # The lower the uniformity factor, the more randomness
            random_variation = (1.0 - UNIFORM_FACTOR) * random.uniform(0.5, 1.5)
            distance = base_distance * (1.0 + (1.0 - UNIFORM_FACTOR) * (random_variation - 1.0))
        else:
            distance = base_distance
        
        # Calculate point coordinates
        x = center_x + distance * np.cos(angle)
        y = center_y + distance * np.sin(angle)
        utm_points.append((x, y))
    
    # Create the polygon in UTM
    utm_polygon = Polygon(utm_points)
    
    # Calculate the area of the generated polygon
    initial_area_m2 = utm_polygon.area
    
    # Scale the polygon to match the target area
    scaling_factor = math.sqrt(target_area_m2 / initial_area_m2)
    
    # Apply scaling: translate to origin, scale, translate back
    def scale_around_center(x, y):
        dx = x - center_x
        dy = y - center_y
        return center_x + dx * scaling_factor, center_y + dy * scaling_factor
    
    scaled_utm_polygon = transform(lambda x, y: scale_around_center(x, y), utm_polygon)
    
    # Transform the UTM polygon back to WGS84 (EPSG:4326)
    wgs84_polygon = transform(transformer_from_utm.transform, scaled_utm_polygon)
    
    # Return the WKT representation
    return wgs84_polygon.wkt

## USER DEFINED GLOBAL VARIABLES

- https://observablehq.com/@jake-low/bounding-box-to-geojson-polygon

In [9]:
# INPUT_BBOX = [11.015629, 55.128649, 24.199222, 69.380313] ## Sweden
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305] # MUNICH
INPUT_BBOX_WKT_STRING = (box(*INPUT_BBOX, ccw=True)).wkt
INPUT_BBOX_WKT_STRING

'POLYGON ((11.723194444453823 48.06152777781623, 11.723194444453823 48.24819444448305, 11.360694444453532 48.24819444448305, 11.360694444453532 48.06152777781623, 11.723194444453823 48.06152777781623))'

In [7]:
calculateArea(INPUT_BBOX_WKT_STRING) 

559.85

## BM 1 - POLYGON WITH FIXED AREA and VARYING NUMBER OF POINTS

In [16]:
NUM_OF_POINTS = 100
AREA_IN_km = 100 
UNIFORM_FACTOR = 0.5

for POINTS in range(3, NUM_OF_POINTS, 5):
    polygon = genPolygonExp(INPUT_BBOX, POINTS, AREA_IN_km, UNIFORM_FACTOR)
    print(f"Polygon Points: {POINTS} \n\n{polygon}\n")
    displayOSM(POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

Polygon Points: 98 

POLYGON ((11.607882719859978 48.1535253383936, 11.615532304409532 48.15649516533572, 11.610589636216796 48.15933852330025, 11.614594214830465 48.16274534654376, 11.616648337082799 48.16627502411329, 11.612437818332129 48.168852890855, 11.60987427138615 48.17156454873001, 11.614836840412556 48.17639802406453, 11.608931690919675 48.17817703509626, 11.610043276153576 48.18238677130921, 11.602494106223906 48.18301114228048, 11.603792000993346 48.187715787892955, 11.608327110043506 48.19500174665774, 11.587160269228193 48.18591743230406, 11.591822414226806 48.19381884190237, 11.587359250902166 48.19528993619738, 11.577358606939827 48.190948863656736, 11.577226461387204 48.19638332278895, 11.575687105515582 48.20125189696492, 11.56821491064569 48.197703170811295, 11.562727718250095 48.19603822220144, 11.562416699787676 48.20612314297821, 11.559269165168551 48.21299883893081, 11.551421394669598 48.20203655343801, 11.546434350687015 48.197623746823346, 11.542361383481092 4

## BM 2 - POLYGON WITH FIXED NUMBER OF POINTS and VARYING AREA 

In [15]:
NUM_OF_POINTS = 50
UNIFORM_FACTOR = 0.4

for AREA_IN_km in np.arange(1, calculateArea(INPUT_BBOX_WKT_STRING), 5):
    polygon = genPolygonExp(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR)
    print(f"Polygon Area: {calculateArea(polygon):.2f}km² \n\n{polygon}\n")
    displayOSM(NUM_OF_POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

Polygon Area: 556.00km² 

POLYGON ((11.715984342011092 48.15102005344512, 11.69562107513517 48.16442028314168, 11.694096495835035 48.17741675165446, 11.743270511027728 48.20294218353598, 11.720652363335915 48.21535079029797, 11.665856943119604 48.21086422281353, 11.660341977801624 48.224256157730515, 11.668819933728408 48.250467425124725, 11.628669337444947 48.239580108351674, 11.638346590432006 48.28028811297423, 11.617101815942583 48.29293558680058, 11.576638722581944 48.2568495243196, 11.560322468581667 48.28018166390664, 11.536736979188616 48.279288142204905, 11.515005330121395 48.271200443759255, 11.491070461272484 48.27338504933485, 11.479916975344567 48.25166209543224, 11.444911416987026 48.265298809585275, 11.443078423563458 48.240675037660765, 11.413754805253129 48.24106256670636, 11.404844285303511 48.226389689439486, 11.394113753881895 48.213685847814844, 11.412644633965469 48.19260122610435, 11.389824825599652 48.184696862202955, 11.335176034048205 48.1769828624098, 11.3866

## BM 3 - POLYGON WITH SINGLE HOLE FIXED AREA and VARYING NUMBER OF POINTS

### Generator

In [11]:
def genPolygonWithHoles(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR, NUMBER_OF_DISJOINT_HOLES):
    """
    Generate a polygon with specified area in square kilometers and containing disjoint holes
    
    Parameters:
    INPUT_BBOX: WKT string representing the bounding box
    NUM_OF_POINTS: Number of points in the outer polygon
    AREA_IN_km: Target area in square kilometers for the entire polygon (including holes)
    UNIFORM_FACTOR: Controls shape regularity (0.0: very irregular, 1.0: perfectly regular)
    NUMBER_OF_DISJOINT_HOLES: Number of holes to create within the polygon
    
    Returns:
    WKT string representing the generated polygon with holes in EPSG:4326
    """
    # Parse the bounding box
    if type(INPUT_BBOX) == str:
        bbox = wkt.loads(INPUT_BBOX)
        bbox_bounds = bbox.bounds
        min_x, min_y, max_x, max_y = bbox_bounds
    else:
        min_x, min_y, max_x, max_y = INPUT_BBOX
        bbox = wkt.loads((box(*INPUT_BBOX, ccw=True)).wkt)
    
    # Compute the centroid for UTM zone determination
    centroid_x = (min_x + max_x) / 2
    centroid_y = (min_y + max_y) / 2
    
    # Determine UTM zone
    utm_zone = int((centroid_x + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    # Create transformers for WGS84 <-> UTM conversion
    transformer_to_utm = Transformer.from_crs("EPSG:4326", proj_string, always_xy=True)
    transformer_from_utm = Transformer.from_crs(proj_string, "EPSG:4326", always_xy=True)
    
    # Transform the bounding box to UTM
    utm_bbox = transform(transformer_to_utm.transform, bbox)
    utm_bounds = utm_bbox.bounds
    utm_min_x, utm_min_y, utm_max_x, utm_max_y = utm_bounds
    
    # Calculate the center point of the bounding box in UTM
    center_x = (utm_min_x + utm_max_x) / 2
    center_y = (utm_min_y + utm_max_y) / 2
    
    # Define how much of the total area will be allocated to holes (30% for holes)
    hole_area_percentage = 0.3
    solid_area_percentage = 1.0 - hole_area_percentage
    
    # Calculate total area needed for the target visible area
    target_area_m2 = AREA_IN_km * 1_000_000  # Convert km² to m²
    total_area_needed = target_area_m2 / solid_area_percentage
    
    # For a regular polygon, calculate the radius
    n = NUM_OF_POINTS
    regular_radius = math.sqrt(total_area_needed / ((n/4) * math.sin(2 * math.pi / n)))
    
    # Generate points for the outer polygon
    angles = np.linspace(0, 2 * np.pi, NUM_OF_POINTS, endpoint=False)
    
    utm_points = []
    for angle in angles:
        # Start with perfectly regular distance (radius)
        base_distance = regular_radius
        
        # Add randomness based on the uniformity factor
        if UNIFORM_FACTOR < 1.0:
            # The lower the uniformity factor, the more randomness
            random_variation = (1.0 - UNIFORM_FACTOR) * random.uniform(0.5, 1.5)
            distance = base_distance * (1.0 + (1.0 - UNIFORM_FACTOR) * (random_variation - 1.0))
        else:
            distance = base_distance
        
        # Calculate point coordinates
        x = center_x + distance * np.cos(angle)
        y = center_y + distance * np.sin(angle)
        utm_points.append((x, y))
    
    # Create the outer polygon in UTM
    outer_utm_polygon = Polygon(utm_points)
    
    # Calculate the area of the generated polygon
    initial_area_m2 = outer_utm_polygon.area
    
    # Scale the polygon to match the target total area
    scaling_factor = math.sqrt(total_area_needed / initial_area_m2)
    
    # Apply scaling: translate to origin, scale, translate back
    def scale_around_center(x, y):
        dx = x - center_x
        dy = y - center_y
        return center_x + dx * scaling_factor, center_y + dy * scaling_factor
    
    scaled_outer_utm_polygon = transform(lambda x, y: scale_around_center(x, y), outer_utm_polygon)
    
    # Create the holes
    holes = []
    if NUMBER_OF_DISJOINT_HOLES > 0:
        # Determine size of each hole
        hole_total_area = total_area_needed * hole_area_percentage
        area_per_hole = hole_total_area / NUMBER_OF_DISJOINT_HOLES
        
        # Calculate the maximum radius for holes to maintain disjointness
        max_hole_radius = regular_radius * scaling_factor * 0.2  # Limit to 40% of polygon radius
        
        # Calculate radius for each hole based on area
        hole_points = max(5, NUM_OF_POINTS // 2)  # Use at least 5 points for each hole
        hole_radius = min(
            math.sqrt(area_per_hole / ((hole_points/4) * math.sin(2 * math.pi / hole_points))),
            max_hole_radius
        )
        
        # Calculate positions for the holes
        # For simplicity, we'll place them in a circle around the center
        if NUMBER_OF_DISJOINT_HOLES == 1:
            # Single hole in the center
            hole_centers = [(center_x, center_y)]
        else:
            # Multiple holes arranged in a circle
            hole_placement_radius = regular_radius * scaling_factor * 0.5  # Place at 50% of the way to edge
            hole_angles = np.linspace(0, 2 * np.pi, NUMBER_OF_DISJOINT_HOLES, endpoint=False)
            hole_centers = [
                (
                    center_x + hole_placement_radius * np.cos(angle),
                    center_y + hole_placement_radius * np.sin(angle)
                )
                for angle in hole_angles
            ]
        
        # Create each hole
        for i, (hcx, hcy) in enumerate(hole_centers):
            hole_angles = np.linspace(0, 2 * np.pi, hole_points, endpoint=False)
            hole_points_coords = []
            
            # Generate points for this hole
            for angle in hole_angles:
                # Start with perfectly regular distance
                base_distance = hole_radius
                
                # Add randomness based on the uniformity factor
                if UNIFORM_FACTOR < 1.0:
                    random_variation = (1.0 - UNIFORM_FACTOR) * random.uniform(0.7, 1.3)
                    distance = base_distance * (1.0 + (1.0 - UNIFORM_FACTOR) * (random_variation - 1.0))
                else:
                    distance = base_distance
                
                # Calculate point coordinates
                x = hcx + distance * np.cos(angle)
                y = hcy + distance * np.sin(angle)
                hole_points_coords.append((x, y))
            
            # Create the hole polygon
            hole = Polygon(hole_points_coords)
            
            # Ensure the hole is contained within the outer polygon
            if hole.within(scaled_outer_utm_polygon):
                holes.append(hole)
    
    # Create the final polygon with holes
    if holes:
        polygon_with_holes = Polygon(scaled_outer_utm_polygon.exterior.coords, 
                                    [h.exterior.coords for h in holes])
    else:
        polygon_with_holes = scaled_outer_utm_polygon
    
    # Verify the final area to ensure it matches the target
    final_area_m2 = polygon_with_holes.area
    
    # Fine-tune scaling to get exact area if needed
    fine_tune_factor = math.sqrt(target_area_m2 / final_area_m2)
    
    def fine_tune_scale(x, y):
        dx = x - center_x
        dy = y - center_y
        return center_x + dx * fine_tune_factor, center_y + dy * fine_tune_factor
    
    final_utm_polygon = transform(lambda x, y: fine_tune_scale(x, y), polygon_with_holes)
    
    # Transform the UTM polygon back to WGS84 (EPSG:4326)
    wgs84_polygon = transform(transformer_from_utm.transform, final_utm_polygon)
    
    # Return the WKT representation
    return wgs84_polygon.wkt

In [12]:
NUM_OF_POINTS = 50
AREA_IN_km = 100.0
UNIFORM_FACTOR = 0.4
NUMBER_OF_DISJOINT_HOLES = 1
  
for POINTS in range(3, NUM_OF_POINTS, 5):
    polygon = genPolygonWithHoles(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR, NUMBER_OF_DISJOINT_HOLES)
    print(f"Polygon Points: {POINTS} \n\n{polygon}\n\n")
    displayOSM(POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

Polygon Points: 48 

POLYGON ((11.604490131442134 48.153602313268465, 11.629929139020744 48.160408603148376, 11.611994757700318 48.16531962901269, 11.62813659578671 48.175532221556885, 11.615035712734167 48.17966610078022, 11.598002266466656 48.18022078212328, 11.61224105330288 48.19607711562728, 11.599128121095953 48.19795102760054, 11.581891737398882 48.19382618380696, 11.574862398639475 48.19757467956687, 11.568346258332397 48.20315006482416, 11.561357205161483 48.2116090618022, 11.550521795825995 48.212284354912555, 11.540268566249983 48.20007516412524, 11.529672226102225 48.208762634034755, 11.52251593913789 48.200746030020326, 11.511059115589244 48.20343503064388, 11.500421569250063 48.20247878069173, 11.505793063943727 48.18654939774834, 11.494187595881748 48.18724908225179, 11.479139770572152 48.18783877964618, 11.491501127804248 48.17514510789942, 11.474157466935225 48.17478163622872, 11.459365084444244 48.17117806757345, 11.467239854284312 48.16300743633601, 11.45532594194054

## BM 4 - POLYGON WITH TWO HOLES WITH AREA AND VARYING NUMBER OF POINTS

In [13]:
NUM_OF_POINTS = 50
AREA_IN_km = 100.0
UNIFORM_FACTOR = 0.4
NUMBER_OF_DISJOINT_HOLES = 2
  
for POINTS in range(3, NUM_OF_POINTS, 10):
    polygon = genPolygonWithHoles(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR, NUMBER_OF_DISJOINT_HOLES)
    print(f"Polygon Points: {POINTS} \n\n{polygon}\n\n")
    displayOSM(POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

Polygon Points: 43 

POLYGON ((11.614218481957497 48.15338131673646, 11.632481870588858 48.16056520887552, 11.616685352695923 48.16601250092055, 11.59849326049619 48.16845136509775, 11.59976367067751 48.17449383088427, 11.59990987861444 48.18108333175801, 11.605377322578805 48.19205187851333, 11.59777410824574 48.19692919058208, 11.593651650742334 48.2053384724686, 11.575082827497353 48.19786241597084, 11.567792840761959 48.20212891277394, 11.55856115120518 48.203321993330256, 11.550515133220475 48.21223812730127, 11.539771100580682 48.21124155231166, 11.532478695360314 48.19679919483561, 11.518106774653154 48.210943568273365, 11.507119812813736 48.20954079843504, 11.504704886642468 48.19762406020396, 11.496756467599853 48.194357394821246, 11.491635608409167 48.18895904963945, 11.490646202605241 48.18185888469421, 11.46694096990294 48.184877818261235, 11.462689902506884 48.178107070416516, 11.465165533414 48.170048236003105, 11.455934015845942 48.164210075666496, 11.466043932520604 48.

In [14]:
NUMBER_OF_DISJOINT_HOLES = 5
  
for POINTS in range(3, NUM_OF_POINTS, 10):
    polygon = genPolygonWithHoles(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR, NUMBER_OF_DISJOINT_HOLES)
    print(f"Polygon Points: {POINTS} \n\n{polygon}\n\n")
    displayOSM(POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

Polygon Points: 43 

POLYGON ((11.623230613214258 48.15317585400276, 11.638027601702552 48.160905230679795, 11.607725602883376 48.16468882630597, 11.621833007026984 48.17402730486817, 11.621179299367652 48.18174590687953, 11.59922047577409 48.180771617133, 11.59373478697922 48.185221682815566, 11.596298640586166 48.19581562895292, 11.595723832191137 48.207366343255266, 11.572380307028022 48.1943344257449, 11.572879902213762 48.21151372945075, 11.55830503452271 48.20256275730783, 11.550017476878478 48.20878493401226, 11.540204412773178 48.20151534115404, 11.527566884239404 48.21773389696838, 11.521893159922596 48.20218664191338, 11.511946699437244 48.20205909701936, 11.50886443123351 48.19290861496385, 11.488867443354223 48.20117136400492, 11.491239151339348 48.18922468188757, 11.460586225537494 48.19747576780002, 11.46750958397453 48.184652587639185, 11.463408379282827 48.17789876800748, 11.44405823535203 48.17415795431052, 11.44384906701615 48.16549430368283, 11.473802628712239 48.156

## BM 5